## 사전 준비

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

# New Train Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# importing packages

# 기본 패키지

import pandas as pd
import numpy as np

# 시각화 패키지

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns

# 데이터 분석 패키지

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/KUBIC_DL_project/data/new_train.csv')

In [ ]:
train.head()

,ID,매물확인방식,보증금,월세,전용면적,해당층,총층,방향,방수,욕실수,주차가능여부,총주차대수,관리비,중개사무소,제공플랫폼,게재일,허위매물여부,NA개수
0,TRAIN_0000,현장확인,402500000.0,470000,26.4,1.0,15.0,서향,1.0,1.0,가능,40.0,96,t93Nt6I2I0,B플랫폼,2024-10-09,0,2
1,TRAIN_0001,현장확인,170500000.0,200000,26.4,3.0,4.0,남동향,2.0,1.0,불가능,0.0,0,q39iV5J4E6,D플랫폼,2024-12-26,0,2
2,TRAIN_0002,전화확인,114000000.0,380000,26.4,2.0,3.0,동향,1.0,1.0,불가능,0.0,0,b03oE4G3F6,A플랫폼,2024-11-28,0,2
3,TRAIN_0003,현장확인,163500000.0,30000,36.3,3.0,9.0,남동향,2.0,1.0,가능,13.0,10,G52Iz8V2B9,A플랫폼,2024-11-26,0,0
4,TRAIN_0004,현장확인,346000000.0,530000,26.4,3.0,3.0,동향,2.0,1.0,불가능,0.0,0,N45gM0M7R0,B플랫폼,2024-06-25,1,2


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2448 entries, 0 to 2447
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      2448 non-null   object 
 1   매물확인방식  2448 non-null   object 
 2   보증금     2448 non-null   float64
 3   월세      2448 non-null   int64  
 4   전용면적    2448 non-null   float64
 5   해당층     2448 non-null   float64
 6   총층      2448 non-null   float64
 7   방향      2448 non-null   object 
 8   방수      2448 non-null   float64
 9   욕실수     2448 non-null   float64
 10  주차가능여부  2448 non-null   object 
 11  총주차대수   2448 non-null   float64
 12  관리비     2448 non-null   int64  
 13  중개사무소   2448 non-null   object 
 14  제공플랫폼   2448 non-null   object 
 15  게재일     2448 non-null   object 
 16  허위매물여부  2448 non-null   int64  
 17  NA개수    2448 non-null   int64  
dtypes: float64(7), int64(4), object(7)
memory usage: 344.4+ KB


# Test Dataset

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/KUBIC_DL_project/data/test.csv')

In [ ]:
test.head()

,ID,매물확인방식,보증금,월세,전용면적,해당층,총층,방향,방수,욕실수,주차가능여부,총주차대수,관리비,중개사무소,제공플랫폼,게재일
0,TEST_000,서류확인,223000000.0,530000,NaN,5.0,5.0,남향,1.0,1.0,불가능,NaN,7,Z86Th6S3K5,D플랫폼,2024-10-21
1,TEST_001,서류확인,150500000.0,590000,30.26,7.0,11.0,서향,1.0,1.0,불가능,16.0,11,G52Iz8V2B9,D플랫폼,2023-09-02
2,TEST_002,현장확인,47000000.0,200000,41.50,2.0,3.0,남서향,2.0,1.0,불가능,NaN,0,N45gM0M7R0,B플랫폼,2024-04-03
3,TEST_003,서류확인,133000000.0,250000,31.35,5.0,6.0,남향,2.0,1.0,가능,NaN,5,C41wx1K6U9,B플랫폼,2024-09-19
4,TEST_004,현장확인,108000000.0,380000,23.14,2.0,3.0,남향,1.0,1.0,불가능,NaN,0,Z68ZJ6F6L4,A플랫폼,2024-11-10


In [ ]:
# test 데이터에 NA개수 컬럼 추가 (각 행의 결측치 수)
test['NA개수'] = test.isnull().sum(axis=1)

In [ ]:
test.head()

,ID,매물확인방식,보증금,월세,전용면적,해당층,총층,방향,방수,욕실수,주차가능여부,총주차대수,관리비,중개사무소,제공플랫폼,게재일,NA개수
0,TEST_000,서류확인,223000000.0,530000,NaN,5.0,5.0,남향,1.0,1.0,불가능,NaN,7,Z86Th6S3K5,D플랫폼,2024-10-21,2
1,TEST_001,서류확인,150500000.0,590000,30.26,7.0,11.0,서향,1.0,1.0,불가능,16.0,11,G52Iz8V2B9,D플랫폼,2023-09-02,0
2,TEST_002,현장확인,47000000.0,200000,41.50,2.0,3.0,남서향,2.0,1.0,불가능,NaN,0,N45gM0M7R0,B플랫폼,2024-04-03,1
3,TEST_003,서류확인,133000000.0,250000,31.35,5.0,6.0,남향,2.0,1.0,가능,NaN,5,C41wx1K6U9,B플랫폼,2024-09-19,1
4,TEST_004,현장확인,108000000.0,380000,23.14,2.0,3.0,남향,1.0,1.0,불가능,NaN,0,Z68ZJ6F6L4,A플랫폼,2024-11-10,1


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 613 entries, 0 to 612
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      613 non-null    object 
 1   매물확인방식  613 non-null    object 
 2   보증금     613 non-null    float64
 3   월세      613 non-null    int64  
 4   전용면적    429 non-null    float64
 5   해당층     563 non-null    float64
 6   총층      611 non-null    float64
 7   방향      613 non-null    object 
 8   방수      611 non-null    float64
 9   욕실수     611 non-null    float64
 10  주차가능여부  613 non-null    object 
 11  총주차대수   438 non-null    float64
 12  관리비     613 non-null    int64  
 13  중개사무소   613 non-null    object 
 14  제공플랫폼   613 non-null    object 
 15  게재일     613 non-null    object 
 16  NA개수    613 non-null    int64  
dtypes: float64(7), int64(3), object(7)
memory usage: 81.5+ KB


# 데이터 전처리

In [ ]:
for df in [train, test]:
    df['게재일'] = pd.to_datetime(df['게재일'])
    df['게재년'] = df['게재일'].dt.year
    df['게재월'] = df['게재일'].dt.month
    df['게재일자'] = df['게재일'].dt.day
    df.drop('게재일', axis=1, inplace=True)

In [ ]:
ID_col = 'ID'
target_col = '허위매물여부'

# 학습에 사용할 feature: ID와 target_col을 제외한 모든 컬럼
features = [col for col in train.columns if col not in [ID_col, target_col]]
target = '허위매물여부'


In [ ]:
# 범주형: 매물확인방식, 방향, 주차가능여부, 중개사무소, 제공플랫폼
categorical_features = ['매물확인방식', '방향', '주차가능여부', '중개사무소', '제공플랫폼']

numeric_features = [col for col in features if col not in categorical_features]

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

In [ ]:
# 수치형 변수 전처리: 결측치 중앙값 대체 + SimpleImputer
# numeric_pipeline = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='median')),
#     ('scaler', StandardScaler())
# ])

# 수치형 변수 전처리: KNNImputer + StandardScaler
numeric_pipeline = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=5)),
    ('scaler', StandardScaler())
])


In [ ]:
# 범주형 변수 전처리: 결측치는 최빈값으로 대체한 후 One-Hot Encoding
categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, numeric_features),
        ('cat', categorical_pipeline, categorical_features)
    ]
)

In [ ]:
# 전처리 후, train 데이터와 test 데이터를 변환
X_train = train[features]
y_train = train[target]

In [ ]:
X_train_preprocessed = preprocessor.fit_transform(X_train)
print("전처리 train 데이터 shape:", X_train_preprocessed.shape)

전처리 train 데이터 shape: (2448, 318)


In [ ]:
X_test = test.drop('ID', axis=1)  # test 데이터에는 target이 없으므로 'ID'만 제외
X_test_preprocessed = preprocessor.transform(X_test)
print("전처리된 test 데이터 shape:", X_test_preprocessed.shape)

전처리된 test 데이터 shape: (613, 318)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from xgboost import XGBClassifier

X_train_split, X_val, y_train_split, y_val = train_test_split(
    X_train_preprocessed, y_train, test_size=0.2, random_state=42, stratify=y_train
)

* Optuna 최적화 적용

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 7.7 MB/s eta 0:00:00


In [ ]:
import optuna
from optuna.samplers import TPESampler
from sklearn.model_selection import StratifiedKFold, cross_val_score
from xgboost import XGBClassifier
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
import numpy as np

In [ ]:
def objective(trial):
    # 하이퍼파라미터 제안
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 30),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'use_label_encoder': False,
        'eval_metric': 'logloss',
        'random_state': 42,
        'n_jobs': -1
    }

    # XGBoost 모델 생성
    model = XGBClassifier(**params)

    # SMOTE와 XGBoost를 포함한 파이프라인 구성
    pipeline = ImbPipeline(steps=[
        ('smote', SMOTE(random_state=42)),
        ('classifier', model)
    ])

    # Stratified 5-Fold 교차 검증 수행
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = cross_val_score(pipeline, X_train_preprocessed, y_train, cv=kf, scoring='accuracy', n_jobs=-1)

    return np.mean(cv_scores)


In [ ]:
# Optuna 스터디 생성 및 최적화 진행
study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42))
study.optimize(objective, n_trials=100, show_progress_bar=True)


print("Best Params:", study.best_params)
print("Best CV Accuracy:", study.best_value)

[I 2025-02-19 10:03:58,687] A new study created in memory with name: no-name-9c7c9e3a-7571-491d-8419-fb6839527b33


  0%|          | 0/100 [00:00<?, ?it/s]

<ipython-input-229-ab44aa9c9e25>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
<ipython-input-229-ab44aa9c9e25>:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
<ipython-input-229-ab44aa9c9e25>:8: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
<ipython-input-229-ab44aa9c9e25>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. Thi

[I 2025-02-19 10:04:11,063] Trial 0 finished with value: 0.964462251158132 and parameters: {'n_estimators': 144, 'max_depth': 29, 'learning_rate': 0.1205712628744377, 'subsample': 0.7993292420985183, 'colsample_bytree': 0.5780093202212182, 'gamma': 1.7699302940633311e-07}. Best is trial 0 with value: 0.964462251158132.


<ipython-input-229-ab44aa9c9e25>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
<ipython-input-229-ab44aa9c9e25>:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
<ipython-input-229-ab44aa9c9e25>:8: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
<ipython-input-229-ab44aa9c9e25>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. Thi

...

[I 2025-02-19 10:11:00,620] Trial 97 finished with value: 0.9603772797462543 and parameters: {'n_estimators': 178, 'max_depth': 23, 'learning_rate': 0.06891145277245278, 'subsample': 0.8998775388456428, 'colsample_bytree': 0.7176348457409082, 'gamma': 0.0009374251655120808}. Best is trial 71 with value: 0.966092400150244.


<ipython-input-229-ab44aa9c9e25>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
<ipython-input-229-ab44aa9c9e25>:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
<ipython-input-229-ab44aa9c9e25>:8: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
<ipython-input-229-ab44aa9c9e25>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. Thi

[I 2025-02-19 10:11:12,570] Trial 98 finished with value: 0.9624180960727851 and parameters: {'n_estimators': 206, 'max_depth': 28, 'learning_rate': 0.03019521785054905, 'subsample': 0.8428289406762857, 'colsample_bytree': 0.7307673819535951, 'gamma': 0.00040501658346456894}. Best is trial 71 with value: 0.966092400150244.


<ipython-input-229-ab44aa9c9e25>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
<ipython-input-229-ab44aa9c9e25>:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
<ipython-input-229-ab44aa9c9e25>:8: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
<ipython-input-229-ab44aa9c9e25>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. Thi

[I 2025-02-19 10:11:14,107] Trial 99 finished with value: 0.9456725512290806 and parameters: {'n_estimators': 163, 'max_depth': 3, 'learning_rate': 0.044563588408193786, 'subsample': 0.8191510849250546, 'colsample_bytree': 0.7429632004435012, 'gamma': 0.0003561827514311876}. Best is trial 71 with value: 0.966092400150244.
Best Params: {'n_estimators': 184, 'max_depth': 5, 'learning_rate': 0.056200541528958876, 'subsample': 0.930316966385177, 'colsample_bytree': 0.7839723921131669, 'gamma': 4.9477557129593586e-06}
Best CV Accuracy: 0.966092400150244


In [ ]:
best_params = study.best_params
best_params.update({'use_label_encoder': False, 'eval_metric': 'logloss', 'random_state': 42, 'n_jobs': -1})
final_model = XGBClassifier(**best_params)

# 최종 파이프라인: SMOTE와 최적의 XGBoost 모델을 포함
final_pipeline = ImbPipeline(steps=[
    ('smote', SMOTE(random_state=42)),
    ('classifier', final_model)
])

final_pipeline.fit(X_train_preprocessed, y_train)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [10:12:17] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Pipeline(steps=[('smote', SMOTE(random_state=42)),
                ('classifier',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=0.7839723921131669, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric='logloss',
                               feature_types=None, gamma=4.9477557129593586e-06,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None,
                               learning_rate=0.056200541528958876, max_bin=None,
                               max_cat_threshold=None, max_cat_to_onehot=None,
                               max_delta_step=None, max_depth=5,
                               max_leaves=None, min_child_weight=None,
                               missing=nan, monotone_constraints=None,
                               multi_strategy=None, n_estimators=184, n_jobs=-1,
                               num_parallel_tree=None, random_state=42, ...))])

In [ ]:
y_val_pred = final_pipeline.predict(X_val)
print(classification_report(y_val, y_val_pred))
print("Accuracy:", accuracy_score(y_val, y_val_pred))


              precision    recall  f1-score   support

           0       0.99      0.99      0.99       430
           1       0.95      0.95      0.95        60

    accuracy                           0.99       490
   macro avg       0.97      0.97      0.97       490
weighted avg       0.99      0.99      0.99       490

Accuracy: 0.9877551020408163


In [ ]:
final_pipeline.fit(X_train_preprocessed, y_train) # 전체 train 데이터로 모델 재학습
X_test = test[features]  # test 데이터의 feature 컬럼은 train과 동일해야 함
test_preds = final_pipeline.predict(X_test_preprocessed)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [10:12:42] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [ ]:
submission = pd.DataFrame({
    'ID': test[ID_col],
    '허위매물여부': test_preds
})

submission.to_csv('/content/drive/MyDrive/KUBIC_DL_project/submissions/submission_xgb_new_8.csv', index=False)

최종 성능 : 0.8511029412

